In [1]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps 
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import requests
import json
#import api_key

from census import Census
from us import states
from config import(census_key)

c = Census(census_key , year=2013)



In [3]:

stops = os.path.join("..","Resources" ,"stops.csv")
stops_public = pd.read_csv(stops, header=0)
stops_public.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,corner_placement,stop_position,on_street,at_street,heading
0,243,243,RIO GRANDE/15TH,000606 15Th & Rio Grande,30.278966,-97.746339,NaN,https://www.capmetro.org/stopdetail/index.php?...,0,NaN,NaN,0.0,NW,F,15Th,Rio Grande,288.0
1,466,466,8740 LAMAR/PAYTON GIN,008740 Lamar & Payton Gin,30.356809,-97.701066,NaN,https://www.capmetro.org/stopdetail/index.php?...,0,NaN,NaN,0.0,SW,M,Lamar,Payton Gin,227.0
2,467,467,FAIRFIELD STATION (SB),008630 Lamar & Fairfield,30.355296,-97.703128,NaN,https://www.capmetro.org/stopdetail/index.php?...,0,NaN,NaN,0.0,SW,F,Lamar,Fairfield,230.0
3,468,468,LAMAR/THURMOND,008400 Lamar & Thurmond,30.353139,-97.706082,NaN,https://www.capmetro.org/stopdetail/index.php?...,0,NaN,NaN,0.0,SW,F,Lamar,Thurmond,228.0
4,469,469,8320 LAMAR/MEADOWLARK,008320 Lamar & Meadowlark,30.352256,-97.707295,NaN,https://www.capmetro.org/stopdetail/index.php?...,0,NaN,NaN,0.0,NW,N,Lamar,Meadowlark,228.0


In [17]:
# census_key ="062995e11366521d379e73701d852b9965e4c9f9"
# url="https://api.census.gov/data/2018/acs/acs1?"
# response = requests.get(url).json

#Run census search to retrieve data on all zip codes(2013 acs5 census)
census_data = c.acs5.get(("B01003_001E","B17001_002E"),
                         {'for': 'zip code tabulation area:*'})
#convert to datafame 
census_df = pd.DataFrame(census_data)

#Column Reordering 
census_df = census_df.rename(columns ={"B01003_001E":"Population",
                                       "B17001_002E":"Poverty_count",
                                       "zip code tabulation area":"Zipcode"} )
#Add in poverty rate (povetry count /population )
census_df['Poverty_rate'] = census_df['Poverty_count'].astype(int)/census_df['Population'].astype(int) *100
 
#delete Povetry count 
del census_df['Poverty_count']

#final dataFrame
print(len(census_df))
census_df.head()



33120


,Population,Zipcode,Poverty_rate
0,22121.0,01832,10.903666
1,8295.0,01833,2.302592
2,6675.0,01834,2.187266
3,13527.0,01835,8.523693
4,4547.0,01840,39.542556


In [18]:
# perform Api calls 
api_key = "8469ce8835d24f5f832bfd64ce9d0e2e"
url="https://geoservices.tamu.edu/Services/Geocode/WebService/GeocoderWebServiceHttpNonParsedDetailed_V04_01.aspx?"
# zip_code = 73301
city = "Austin"
query_url = f"{url}&city={city}&zip={zip}&apiKey={api_key}&format=json&census=true&censusYear=2018&notStore=false&version=4.01"
response = requests.get(query_url)



In [19]:
response.json()

{'version': '4.10',
 'TransactionId': '8d87c3de-25d9-4a51-a10f-1da4dd1d3961',
 'Version': '4.1',
 'QueryStatusCodeValue': '200',
 'FeatureMatchingResultType': 'BrokenTie',
 'FeatureMatchingResultCount': '2',
 'TimeTaken': '0.0468209',
 'ExceptionOccured': 'False',
 'Exception': '',
 'InputAddress': {'StreetAddress': 'Austin &lt;clas',
  'City': 'Austin',
  'State': '',
  'Zip': '&lt;clas'},
 'OutputGeocodes': [{'OutputGeocode': {'Latitude': '29.8870240883788',
    'Longitude': '-96.2779074606308',
    'NAACCRGISCoordinateQualityCode': '12',
    'NAACCRGISCoordinateQualityType': 'CountyCentroid',
    'MatchScore': '40',
    'MatchType': 'Relaxed;Soundex',
    'FeatureMatchingResultType': 'BrokenTie',
    'FeatureMatchingResultCount': '2',
    'FeatureMatchingGeographyType': 'County',
    'RegionSize': '1700026133.08375',
    'RegionSizeUnits': 'Meters',
    'MatchedLocationType': 'LOCATION_TYPE_USPS_ZIP_CODE',
    'ExceptionOccured': 'False',
    'Exception': '',
    'ErrorMessage': ''}